In [ ]:
# Imports
import os

# Ensure notebook is running from src/ dir
_pwd = os.path.realpath('.').split(os.sep)
if 'src' in _pwd:
	while _pwd[-1] != 'src':
		_pwd.pop()
	os.chdir(os.sep.join(_pwd))
print(os.path.realpath('.'))

# Local project requirements
from model.tournesol_api import TournesolAPI, get

In [ ]:
# PARAMETERS
TOURNESOL_API=TournesolAPI(input('JWT (example: "Bearer xxxxxxxxx")'))
TOURNESOL_API.loadCache(f"../data/Tournesol_API_cache-{TOURNESOL_API.username}.json.gz")

In [ ]:
# Load dataset
rate_later = TOURNESOL_API.getMyRateLater()
print('Total rate later:', len(rate_later))
filtered = [r for r in rate_later 
    if get(r, 0, 'collective_rating', 'n_contributors') > 0 and (get(r, 0, 'individual_rating', 'n_comparisons') == 0)
]
print('Rate later with other contributors:', len(filtered))

In [ ]:
# Sort by:
# - number of total contributors (smallest first)
# - number of total comparisons (highest first)
# - number of views (largest first)
def sortkey(vdata) -> tuple[int, int, int, int]:
	return (
		get(vdata, 0, 'collective_rating', 'n_contributors'),
		-get(vdata, 0, 'collective_rating', 'n_comparisons'),
		-get(vdata, -100, 'collective_rating', 'tournesol_score'),
		#-get(vdata, 0, 'entity', 'metadata', 'views'),
	)

filtered.sort(key=sortkey)

print('Rank  url                                                                 (n_contributors, n_comparisons, views)')
for r in range(0, len(filtered)-1, 2):
	print(f"{int(r/2)+1:3d}. https://tournesol.app/comparison?uidA={filtered[r]['entity']['uid']}&uidB={filtered[r+1]['entity']['uid']}", 
		(get(filtered[r], 0, 'collective_rating', 'n_contributors'), get(filtered[r], 0, 'collective_rating', 'n_comparisons'), get(filtered[r], 0, 'entity', 'metadata', 'views')),
		(get(filtered[r+1], 0, 'collective_rating', 'n_contributors'), get(filtered[r+1], 0, 'collective_rating', 'n_comparisons'), get(filtered[r+1], 0, 'entity', 'metadata', 'views'))
	)
